In [2]:
from collections import defaultdict
import math
from math import sqrt, sin, cos
import random

import numpy as np

EPS = 1e-8

def same(a, b):
    return abs(a - b) < EPS

def same_points(a, b):
    return same(a[0], b[0]) and same(a[1], b[1]) and same(a[2], b[2])

def rand_rotation_matrix(deflection=1.0, randnums=None):
    """
    Creates a random rotation matrix.

    deflection: the magnitude of the rotation. For 0, no rotation; for 1, competely random
    rotation. Small deflection => small perturbation.
    randnums: 3 random numbers in the range [0, 1]. If `None`, they will be auto-generated.
    """
    # from http://www.realtimerendering.com/resources/GraphicsGems/gemsiii/rand_rotation.c

    if randnums is None:
        randnums = np.random.uniform(size=(3,))

    theta, phi, z = randnums

    theta = theta * 2.0*deflection*np.pi  # Rotation about the pole (Z).
    phi = phi * 2.0*np.pi  # For direction of pole deflection.
    z = z * 2.0*deflection  # For magnitude of pole deflection.

    # Compute a vector V used for distributing points over the sphere
    # via the reflection I - V Transpose(V).  This formulation of V
    # will guarantee that if x[1] and x[2] are uniformly distributed,
    # the reflected points will be uniform on the sphere.  Note that V
    # has length sqrt(2) to eliminate the 2 in the Householder matrix.

    r = np.sqrt(z)
    Vx, Vy, Vz = V = (
        np.sin(phi) * r,
        np.cos(phi) * r,
        np.sqrt(2.0 - z)
        )

    st = np.sin(theta)
    ct = np.cos(theta)

    R = np.array(((ct, st, 0), (-st, ct, 0), (0, 0, 1)))

    # Construct the rotation matrix  ( V Transpose(V) - I ) R.

    M = (np.outer(V, V) - np.eye(3)).dot(R)
    return M

def cyclic_permutations(points):
    new_points = []
    for p in points:
        new_points.append(p)
        new_points.append([p[1], p[2], p[0]])
        new_points.append([p[2], p[0], p[1]])
    return new_points


def all_permutations(points):
    new_points = []
    for p in points:
        new_points.extend(cyclic_permutations([p]))
        new_points.extend(cyclic_permutations([[p[1], p[0], p[2]]]))
    return new_points


def plus_minus(points):
    new_points = []
    for p in points:
        mults = []
        for i in range(len(p)):
            if same(p[i], 0):
                mults.append([1])
            else:
                mults.append([1, -1])
        for i in mults[0]:
            for j in mults[1]:
                for k in mults[2]:
                    new_points.append([p[0] * i, p[1] * j, p[2] * k])
    return new_points

def conf1(points):
    return plus_minus(cyclic_permutations(points))


def conf2(points):
    return plus_minus(all_permutations(points))

# def rotation_matrix(axis, theta):
#     """
#     Return the rotation matrix associated with counterclockwise rotation about
#     the given axis by theta radians.
#     """
#     axis = np.asarray(axis)
#     theta = np.asarray(theta)
#     axis = axis / sqrt(np.dot(axis, axis))
#     a = cos(theta / 2.0)
#     b, c, d = -axis * sin(theta / 2.0)
#     aa, bb, cc, dd = a * a, b * b, c * c, d * d
#     bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
#     return np.array([ \
#             [aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)], \
#             [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)], \
#             [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

# def simple_rotate(points, axis, angle):
#     rm = rotation_matrix(axis, angle)
#     new_points = []
#     for p in points:
#         new_points.append(np.dot(rm, p))
#     return new_points

# def rotate1(points):
#     new_points = copy.deepcopy(points)
#     for i in range(5):
#         for k in range(1, 29, 8):
#             rotated1 = simple_rotate(points[:i], points[i], np.pi * k / 15.0)
#             rotated2 = simple_rotate(points[i+1:], points[i], np.pi * k / 15.0)
#             new_points.extend(rotated1)
#             new_points.extend(rotated2)
#     return new_points

def distance(a, b):
    return sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2 + (a[2] - b[2]) ** 2)

def sphere_distance(a, b):
    mult = a[0] * b[0] + a[1] * b[1] + a[2] * b[2]
    mult = max(min(mult, 1), -1)
    return math.acos(mult)

def gen_icosidodecahedron():
    phi = (1.0 + sqrt(5)) / 2
    a, b, c = 0.5, phi / 2, (1.0 + phi) / 2
    points = conf1([[a, b, c]])
    points.extend(conf1([[0.0, 0.0, phi]]))
    return points

def set_radius(points, radius):
    for i in range(len(points)):
        coef = radius / distance(points[i], [0.0, 0.0, 0.0])
        for j in range(len(points[i])):
            points[i][j] = points[i][j] * coef
    #print(points, file=sys.stderr)
    return points

def uniq_points(points):
    print("before uniq points:", len(points))
    for_remove = set()
    for i in range(len(points)):
        if i not in for_remove:
            for j in range(i + 1, len(points)):
                if j not in for_remove and same_points(points[i], points[j]):
                    for_remove.add(j)
    filtered_points = []
    for i in range(len(points)):
        if i not in for_remove:
            filtered_points.append(points[i])
    print("after uniq points:", len(filtered_points))
    return filtered_points

def add_opposite_points(points):
    opposite_points = dict()
    i = 0
    while i < len(points):
        if i not in opposite_points:
            for j in range(i + 1, len(points)):
                if same_points(points[i], [-x for x in points[j]]):
                    opposite_points[i] = j
                    opposite_points[j] = i
                    break
            if i not in opposite_points:
                j = len(points)
                points.append([-x for x in points[i]])
                opposite_points[i] = j
                opposite_points[j] = i
        i += 1
    return points, opposite_points

def find_trinities_from_points(points, opposite_points, radius):
    pairs = dict()
    triangle_distance = radius * sqrt(3)
    for i in range(len(points)):
        pairs[i] = set()
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            if  opposite_points[j] not in pairs[opposite_points[i]] and opposite_points[i] not in pairs[opposite_points[j]]:
                d = distance(points[i], points[j])
                if same(d, triangle_distance):
                    pairs[i].add(j)
                    opp = [opposite_points[i], opposite_points[j]]
                    opp.sort()
                    pairs[opp[0]].add(opp[1])
    for i in range(len(points)):
        if not pairs[i]:
            pairs.pop(i)

    trinities = set()
    for i in pairs:
        candidates = list(pairs[i])
        candidates.sort()
        for j in range(len(candidates)):
            for k in range(j + 1, len(candidates)):
                if candidates[j] in pairs and candidates[k] in pairs[candidates[j]]:
                    trinities.add((i, candidates[j], candidates[k]))
                    opp = [opposite_points[i], opposite_points[candidates[j]], opposite_points[candidates[k]]]
                    opp.sort()
                    trinities.add(tuple(opp))
    #print("trinities", trinities, file=sys.stderr)
    return trinities


In [10]:
def find_colors(cur_idx, max_idx, flooded, color_order,
                points, opposite_points, trinities_by_idx, 
                neighbours,
                colors, flow_upper_bound):
    if cur_idx > max_idx:
        mod_color_sets = set()
        for p1_idx, p1 in enumerate(points):
            mod_color_set = [colors[p1_idx] % 5] + sorted([colors[v] % 5 for v in neighbours[p1_idx]])
            mod_color_sets.add(tuple(mod_color_set))
        assert (len(mod_color_sets) == 26)

        return False

    cur_val = flooded[cur_idx]
    if colors[cur_val]:
        return find_colors(cur_idx + 1, max_idx, flooded, color_order,
                           points, opposite_points, trinities_by_idx,
                           neighbours,
                           colors, flow_upper_bound)
    for cur_color in color_order:
        if abs(cur_color) >= flow_upper_bound:
            break
#         coord = points[cur_val]
#         if coord[2] > 0 and cur_color < 0:
#             continue
#         if coord[2] < 0 and cur_color > 0:
            continue
        colors[cur_val], colors[opposite_points[cur_val]] = cur_color, -cur_color
        all_good = True
        also_colored = set()
        to_check = set([cur_val, opposite_points[cur_val]])
        checked_trinities = set()
        while all_good and to_check:
            val_for_check = to_check.pop()
            for t in trinities_by_idx[val_for_check]:
                if not t in checked_trinities: # probably no need for this structure
                    checked_trinities.add(t)
                    zeros_count = 0
                    zeros_vals = []
                    for v in t:
                        if not colors[v]:
                            zeros_count += 1
                            zeros_vals.append(v)
                    if zeros_count < 2:
                        flow = colors[val_for_check] + colors[t[0]] + colors[t[1]]
                        all_good = (zeros_count == 0 and flow == 0)
                        all_good = all_good or (zeros_count != 0 and flow != 0 and abs(flow) < flow_upper_bound)
#                         if zeros_count == 1:
#                             v = zeros_vals[0]
#                             coord2 = points[v]
#                             if coord2[2] > 0 and flow > 0:
#                                 all_good = False
#                             if coord2[2] < 0 and flow < 0:
#                                 all_good = False
                        if not all_good:
                            break
                        elif zeros_count == 1:
                            v = zeros_vals[0]
                            colors[v], colors[opposite_points[v]] = -flow, flow
                            also_colored.add(v)
                            also_colored.add(opposite_points[v])
                            to_check.add(v)
                            to_check.add(opposite_points[v])
        if all_good:
            #print("bound:", flow_upper_bound, "cur idx:", cur_idx, "also_colored:", len(also_colored))
            if find_colors(cur_idx + 1, max_idx, flooded, color_order,
                           points, opposite_points, trinities_by_idx,
                           neighbours,
                           colors, flow_upper_bound):
                return True
        for idx in also_colored:
            colors[idx] = 0
    colors[cur_val], colors[opposite_points[cur_val]] = 0, 0
    return False


In [11]:
points = gen_icosidodecahedron()
radius = 1
points = set_radius(points, radius)

points_rot_m = rand_rotation_matrix()
rotated_points = []
for p in points:
    rotated_points.append(list(np.matmul(p, points_rot_m)))
points = rotated_points

points, opposite_points = add_opposite_points(points)
trinities = find_trinities_from_points(points, opposite_points, radius)

# for p in points:
#     print(p)
# print(opposite_points)
print(len(trinities))

neighbours = []
for p1_idx, p1 in enumerate(points):
    min_dist = None
    p1_neighbours = []
    for p2_idx, p2 in enumerate(points):
        if p2_idx == p1_idx:
            continue
        cur_dist = sphere_distance(p1, p2)
        if min_dist is None:
            min_dist = cur_dist
        if cur_dist < min_dist - EPS:
            min_dist = cur_dist
            p1_neighbours = []
        if abs(min_dist - cur_dist) < EPS:
            p1_neighbours.append(p2_idx)
    assert (len(p1_neighbours) == 4)
    neighbours.append(p1_neighbours)

20


In [12]:
flow_upper_bound = 5
color_order = []
for i in range(1, flow_upper_bound):
# for i in [4,3,2,1]:
# for i in [4,3,2,1,-1,-2,-3,-4]:
    color_order.append(i)
    color_order.append(-i)

colors = dict()
for i in opposite_points:
    colors[i] = 0
trinities_by_idx = dict()
for t in trinities:
    for v in t:
        if v not in trinities_by_idx:
            trinities_by_idx[v] = []
        not_vs = []
        for x in t:
            if x != v:
                not_vs.append(x)
        trinities_by_idx[v].append(tuple(not_vs))

found5 = True
component_count = 0
for seed_vertex in opposite_points:
    if not colors[seed_vertex]:
        component_count += 1
        flooded = []
        should_check = set([seed_vertex])
        to_check = set([seed_vertex])
        checked = set()
        while to_check:
            cur_val = to_check.pop()
            if cur_val not in flooded:
                flooded.append(cur_val)
            checked.add(cur_val)
            done_1_flood = False
            random.shuffle(trinities_by_idx[cur_val])
            for t in trinities_by_idx[cur_val]:
                for v in t:
                    if v not in flooded:
                        if not done_1_flood:
                            flooded.append(v)
                            done_1_flood = True
                        to_check.add(v)
                        should_check.add(v)
            if not to_check:
                to_check = should_check - checked
        max_idx = len(flooded) - 1

        flow_upper_bound = 5
        cur_found5 = find_colors(0, max_idx, flooded, color_order,
                                 points, opposite_points, trinities_by_idx,
                                 neighbours,
                                 colors, flow_upper_bound)
        found5 = found5 and cur_found5
        if not found5:
            break

        flows5 = dict()
        for v in should_check:
            flows5[v] = colors[v]
#             print("color", colors[v], sep='\t')
#         if len(flooded) == len(points):
#             for t in trinities:
#                 print("trinity", t[0], t[1], t[2], sep='\t')
#         print("flow values:", flows5)

print('found5', found5)
print('component_count', component_count)

found5 False
component_count 1
